## Change the root directory steps

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\User\\Python\\MLOps_Projects\\datascienceproject\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\User\\Python\\MLOps_Projects\\datascienceproject'

## Dataclass

In [36]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## Configuration manager

In [37]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [53]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH
                 ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

        # to initiate the data ingestion, need to get all the config details of the data ingestion
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [60]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [61]:
## component - Data Ingestion
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        print("DataIngestion Config Attributes:")
        print("source_URL:", self.config.source_URL)
        print("local_data_file:", self.config.local_data_file)

        
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}" )
        else:
            logger.info(f"File already exists")

    # Extract the zip file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [62]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-01 12:56:23,600: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2024-12-01 12:56:23,602: INFO: common: yaml file: params.yaml loaded succesfully]
[2024-12-01 12:56:23,604: INFO: common: yaml file: schema.yaml loaded succesfully]
[2024-12-01 12:56:23,607: INFO: common: created directory at: artifacts]
[2024-12-01 12:56:23,607: INFO: common: created directory at: artifacts/data_ingestion]
DataIngestion Config Attributes:
source_URL: https://github.com/krishnaik06/datasets/raw/refs/heads/main/winequality-data.zip
local_data_file: artifacts/data_ingestion/data.zip
[2024-12-01 12:56:23,609: INFO: 733697267: File already exists]
